***GENERATED CODE FOR financemodelsloanrepaymentmodel PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Loan_ID', 'transformation_label': 'String Indexer'}], 'feature': 'Loan_ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '3', 'mean': '', 'stddev': '', 'min': 'xqd20160407', 'max': 'xqd20160459', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Loan_ID'}, {'feature_label': 'Loan_ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Loan_ID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'loan_status', 'transformation_label': 'String Indexer'}], 'feature': 'loan_status', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3', 'mean': '', 'stddev': '', 'min': 'COLLECTION_PAIDOFF', 'max': 'COLLECTION_PAIDOFF', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'loan_status'}, {'feature_label': 'loan_status', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('loan_status')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'effective_date', 'transformation_label': 'String Indexer'}], 'feature': 'effective_date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3', 'mean': '', 'stddev': '', 'min': '9/10/2016', 'max': '9/12/2016', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'effective_date'}, {'feature_label': 'effective_date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('effective_date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'due_date', 'transformation_label': 'String Indexer'}], 'feature': 'due_date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3', 'mean': '', 'stddev': '', 'min': '10/11/2016', 'max': '9/24/2016', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'due_date'}, {'feature_label': 'due_date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('due_date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'paid_off_time', 'transformation_label': 'String Indexer'}], 'feature': 'paid_off_time', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3', 'mean': '', 'stddev': '', 'min': '10/13/2016 15:10', 'max': '9/26/2016 11:03', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'paid_off_time'}, {'feature_label': 'paid_off_time', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('paid_off_time')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'education', 'transformation_label': 'String Indexer'}], 'feature': 'education', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3', 'mean': '', 'stddev': '', 'min': 'college', 'max': 'college', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'education'}, {'feature_label': 'education', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('education')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3', 'mean': '', 'stddev': '', 'min': 'male', 'max': 'male', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run financemodelsloanrepaymentmodelHooks.ipynb
try:
	#sourcePreExecutionHook()

	loanpaymentsdata = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/Data/finance models/Loan payments data.csv', 'filename': 'Loan payments data.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(loanpaymentsdata)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run financemodelsloanrepaymentmodelHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(loanpaymentsdata,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Loan_ID", "transformation_label": "String Indexer"}], "feature": "Loan_ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3", "mean": "", "stddev": "", "min": "xqd20160407", "max": "xqd20160459", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Loan_ID"}, {"transformationsData": [{"feature_label": "loan_status", "transformation_label": "String Indexer"}], "feature": "loan_status", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3", "mean": "", "stddev": "", "min": "COLLECTION_PAIDOFF", "max": "COLLECTION_PAIDOFF", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "loan_status"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Principal", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3", "mean": "933.33", "stddev": "115.47", "min": "800", "max": "1000", "missing": "0"}, "updatedLabel": "Principal"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "terms", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3", "mean": "25.0", "stddev": "8.66", "min": "15", "max": "30", "missing": "0"}, "updatedLabel": "terms"}, {"transformationsData": [{"feature_label": "effective_date", "transformation_label": "String Indexer"}], "feature": "effective_date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3", "mean": "", "stddev": "", "min": "9/10/2016", "max": "9/12/2016", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "effective_date"}, {"transformationsData": [{"feature_label": "due_date", "transformation_label": "String Indexer"}], "feature": "due_date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3", "mean": "", "stddev": "", "min": "10/11/2016", "max": "9/24/2016", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "due_date"}, {"transformationsData": [{"feature_label": "paid_off_time", "transformation_label": "String Indexer"}], "feature": "paid_off_time", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3", "mean": "", "stddev": "", "min": "10/13/2016 15:10", "max": "9/26/2016 11:03", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "paid_off_time"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "past_due_days", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3", "mean": "7.67", "stddev": "9.81", "min": "2", "max": "19", "missing": "0"}, "updatedLabel": "past_due_days"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3", "mean": "30.67", "stddev": "5.77", "min": "24", "max": "34", "missing": "0"}, "updatedLabel": "age"}, {"transformationsData": [{"feature_label": "education", "transformation_label": "String Indexer"}], "feature": "education", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3", "mean": "", "stddev": "", "min": "college", "max": "college", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "education"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3", "mean": "", "stddev": "", "min": "male", "max": "male", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run financemodelsloanrepaymentmodelHooks.ipynb
try:
	#sinkPreExecutionHook()

	loanrepaymentsmodelregression = NumtraConnector.fetch(spark, "{'samplefile': '/FileStore/platform/sampleData/61d587543c695e1181ce8fd3/part-00000-eae1dbe7-d142-4ca4-b3d4-c1dd7e01ee30-c000.csv', 'samplecount': 500, 'originalcount': 500, 'orignalKey': '/FileStore/platform/Data/finance models/Loan payments data.csv', 'pathOnly': '/Finance Models/Loan Repayments Model - Regression', 'project_id': '61d572ba3879330c024bfffa', 'parent_id': '61d572ba3879330c024bfffa', 'original_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '61d6b5e13c695e1181ce8ff9', 'field': 'Principal', 'alias': 'Principal', 'type': 'numeric', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '61d6b5e13c695e1181ce8ffa', 'field': 'terms', 'alias': 'terms', 'type': 'numeric', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '61d6b5e13c695e1181ce8ffb', 'field': 'past_due_days', 'alias': 'past_due_days', 'type': 'numeric', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '61d6b5e13c695e1181ce8ffc', 'field': 'age', 'alias': 'age', 'type': 'numeric', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '61d6b5e13c695e1181ce8ffd', 'field': 'Loan_ID_stringindexer', 'alias': 'Loan_ID_stringindexer', 'type': 'numeric', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '61d6b5e13c695e1181ce8ffe', 'field': 'loan_status_stringindexer', 'alias': 'loan_status_stringindexer', 'type': 'numeric', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '61d6b5e13c695e1181ce8fff', 'field': 'effective_date_stringindexer', 'alias': 'effective_date_stringindexer', 'type': 'numeric', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '61d6b5e13c695e1181ce9000', 'field': 'due_date_stringindexer', 'alias': 'due_date_stringindexer', 'type': 'numeric', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '61d6b5e13c695e1181ce9001', 'field': 'paid_off_time_stringindexer', 'alias': 'paid_off_time_stringindexer', 'type': 'numeric', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '61d6b5e13c695e1181ce9002', 'field': 'education_stringindexer', 'alias': 'education_stringindexer', 'type': 'numeric', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '61d6b5e13c695e1181ce9003', 'field': 'Gender_stringindexer', 'alias': 'Gender_stringindexer', 'type': 'numeric', 'position': '10'}], 'actual_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '61d6b5e13c695e1181ce8fee', 'field': 'Loan_ID', 'alias': 'Loan_ID', 'type': 'String', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '61d6b5e13c695e1181ce8fef', 'field': 'loan_status', 'alias': 'loan_status', 'type': 'String', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '61d6b5e13c695e1181ce8ff0', 'field': 'Principal', 'alias': 'Principal', 'type': 'numeric', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '61d6b5e13c695e1181ce8ff1', 'field': 'terms', 'alias': 'terms', 'type': 'numeric', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '61d6b5e13c695e1181ce8ff2', 'field': 'effective_date', 'alias': 'effective_date', 'type': 'date', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '61d6b5e13c695e1181ce8ff3', 'field': 'due_date', 'alias': 'due_date', 'type': 'date', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '61d6b5e13c695e1181ce8ff4', 'field': 'paid_off_time', 'alias': 'paid_off_time', 'type': 'date', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '61d6b5e13c695e1181ce8ff5', 'field': 'past_due_days', 'alias': 'past_due_days', 'type': 'real', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '61d6b5e13c695e1181ce8ff6', 'field': 'age', 'alias': 'age', 'type': 'numeric', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '61d6b5e13c695e1181ce8ff7', 'field': 'education', 'alias': 'education', 'type': 'String', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '61d6b5e13c695e1181ce8ff8', 'field': 'Gender', 'alias': 'Gender', 'type': 'String', 'position': '10'}], 'server': 'https://ai.numtra.com:443', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'LoadRepaymentOutput.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-728a1553-3a4e-4a94-abba-c46b78b333f3-c000.csv', 'results_url': 'http://3.139.69.95:4040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
